In [5]:
import urllib.request as url
import pandas as pd
from geopy.distance import vincenty
import webbrowser
import json
import time
import subprocess
import os
from geopy.geocoders import Nominatim
geolocator = Nominatim()
LAT = 40.7829
LON = 73.9654
global source
source = (LAT,LON)

In [50]:
table_data = pd.read_csv('open_table_data_final.csv',encoding = "ISO-8859-1")
len(table_data)


595

In [51]:
table_data[:5]["lat"]


0    40.745962
1    40.745394
2    40.751123
3    40.748476
4    40.748698
Name: lat, dtype: float64

In [52]:
table_data[:5]["lng"]

0   -73.988622
1   -73.989588
2   -74.002069
3   -74.003339
4   -74.003228
Name: lng, dtype: float64

In [53]:
def calculateDistance(table_data,source):
    distance = []
    #print(source)
    #table_data1 = pd.DataFrame()
    for i in range(len(table_data)):
        lat_dest = table_data.at[i,"lat"]
        lng_dest = table_data.at[i,"lng"]
        dest = (lat_dest,lng_dest)
        distance.append(vincenty(source, dest).miles)
    table_data = table_data.assign(dist = distance)
    #print(table_data.sort_values(by=['price','dist'], ascending=[True,True])[1:5])
    return table_data


In [54]:
#using geopy displacement
def getLatAndLng(place):
    location = geolocator.geocode(place)
    print((location.latitude, location.longitude))
    return [location.latitude,location.longitude]

In [55]:
#using osrm matrix distance
def calculateDistanceOSRM(table_data,source):
    distance = []
    table_temp = pd.DataFrame()
    #data = json.load(url.urlopen("http://router.project-osrm.org/table/v1/drive/-73.987330,40.748711;-74.0134,40.7127?sources=1"))
    for i in range(5):
        lat_dest = table_data.at[i,"lat"]
        lng_dest = table_data.at[i,"lng"]
        json_data = url.urlopen("http://router.project-osrm.org/table/v1/drive/"+str(source[1])+","+str(source[0])+";"+str(lng_dest)+","+str(lat_dest)+"?sources=1")
        data = json.load(json_data)
        calc_distance = round((data['durations'][0][0])*0.000621371,4)
        distance.append(calc_distance)
    
    table_temp = table_data[0:5]
    table_temp = table_temp.assign(dist = distance)
    #table_data = table_data.assign(dist = distance)
    #print(table_data.sort_values(by=['price','dist'], ascending=[True,True])[1:5])
    #return table_data
    return table_temp

In [56]:
def searchForRestaurant(query,table_data):
    table_data_dist_osrm = pd.DataFrame()
    if("near me" in query or "nearme" in query):
        LAT = 40.7829
        LON = 73.9654
        source = (LAT,LON)
    else:
        place = query.split("near")
        loc = getLatAndLng(place)
        LAT = loc[0]
        LON = loc[1]
        source = (LAT,LON)
        table_data_dist = pd.DataFrame()
        table_data_dist = calculateDistance(table_data,source)
        tabel_data_dist_osrm = calculateDistanceOSRM(table_data,source)
        #print(type(table_data_dist_osrm))
    return (table_data_dist,tabel_data_dist_osrm,LAT,LON)

In [57]:
def openMap(src_LAT,src_LON,des_LAT,des_LON):
    webbrowser.open("https://www.openstreetmap.org/directions?engine=graphhopper_car&route="+str(src_LAT)+"%2C"+str(src_LON)+"%3B"+str(des_LAT)+"%2C"+str(des_LON),new=2)
    time.sleep(1)

In [7]:
def call_recommender():
    os.remove('success')
    subprocess.call('run_recommender.bat')
    while(not(os.path.exists('success'))):
        None

In [58]:
table_data_dist = pd.DataFrame()
tabel_data_dist_osrm = pd.DataFrame()
temp = pd.DataFrame()
call_recommender()
print("please enter your query to search")
table_data_dist,tabel_data_dist_osrm,src_LAT,src_LON = searchForRestaurant(input(),table_data)
#print(type(table_data_dist))
#table_data_dist.sort_values(by=['price','dist'], ascending=[True,True])[1:5]
temp = tabel_data_dist_osrm.sort_values(by=['price','dist'], ascending=[True,True])[1:5]
print(temp)
for i in range(4):
    openMap(src_LAT,src_LON,temp.iloc[i]["lat"],temp.iloc[i]["lng"])

please enter your query to search
near bryant park
(40.75374985, -73.9835672012434)
       id                  name          address      city state                       area  postal_code country        phone        lat        lng  price                                      reserve_url                                 mobile_reserve_url                                          image_url    dist
2  115729          Gastromarket     315 10th Ave  New York    NY  New York / Tri-State Area        10001      US  2126958080x  40.751123 -74.002069      3  http://www.opentable.com/single.aspx?rid=115729  http://mobile.opentable.com/opentable/?restId=...  https://www.opentable.com/img/restimages/11572...  0.3815
4  103945             Chop Shop  254 10th Avenue  New York    NY  New York / Tri-State Area        10001      US   2128200333  40.748698 -74.003228      3  http://www.opentable.com/single.aspx?rid=103945  http://mobile.opentable.com/opentable/?restId=...  https://www.opentable.com/img/re

In [59]:
#calculateDistance()
#table_data_dist
#table_data[:5]
#table_data.sort_values(by=['price','dist'], ascending=[True,True])[1:5]
print(temp.iloc[0]["lat"])
print(temp.iloc[0]["lng"])

40.751123
-74.002069
